In [24]:
import sys 
sys.path.insert(0, '../')

In [25]:
from torch.utils.data import DataLoader
import pandas as pd
from common.common import create_folder,H5Recorder
import numpy as np
from torch.utils.data.dataset import Dataset
import os
import torch
import torch.nn as nn
import pytorch_pretrained_bert as Bert

from model import optimiser
import sklearn.metrics as skm
import math
from torch.utils.data.dataset import Dataset
import random
import numpy as np
import torch
import time
from sklearn.metrics import roc_auc_score
from common.common import load_obj
from model.utils import age_vocab
from dataLoader.NextXVisit import NextVisit
from model.NextXVisit import BertForMultiLabelPrediction
import warnings
import random
warnings.filterwarnings(action='ignore')

In [26]:
# seeds
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [22]:
temp = pd.read_parquet('for_behrt.parquet')
temp.shape

FileNotFoundError: [Errno 2] No such file or directory: 'for_behrt.parquet'

In [23]:
temp['visits'] = temp['FRS_diag'].apply(lambda x: [idx for idx, d in enumerate(x) if d == 'SEP'])

NameError: name 'temp' is not defined

In [10]:
temp['visits'] = temp['visits'].apply(lambda x: x[1:-1])
temp['visits'].head()

NameError: name 'temp' is not defined

In [11]:
temp = temp.rename(columns={'ID':'PID'})

NameError: name 'temp' is not defined

In [12]:
import random
temp['t_idx'] = temp['visits'].apply(lambda x: random.randint(0, len(x)-2))

NameError: name 'temp' is not defined

In [13]:
temp['start_idx'] = temp.apply(lambda row: row['visits'][row['t_idx']], axis=1)
temp['stop_idx'] = temp.apply(lambda row: row['visits'][row['t_idx']+1], axis=1)


NameError: name 'temp' is not defined

In [14]:

temp['target'] = temp.apply(lambda row: row['FRS_diag'][row['start_idx']+1:row['stop_idx']], axis=1)


NameError: name 'temp' is not defined

In [11]:
train = temp.iloc[:int(temp.shape[0]*.8),].reset_index(drop=True)
test = temp.iloc[int(temp.shape[0]*.8):,].reset_index(drop=True)

train_s = train.iloc[:int(train.shape[0]*.8),].reset_index(drop=True)
validation = train.iloc[int(train.shape[0]*.8):,].reset_index(drop=True)



In [12]:
train.to_parquet('train_all.parquet')
train_s.to_parquet('train.parquet')
validation.to_parquet('validation.parquet')
test.to_parquet('test.parquet')

In [13]:
file_config = {
    'vocab': 'vocab', # token2idx idx2token
    'train': 'train.parquet',
    'test': 'validation.parquet',
}

optim_config = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

global_params = {
    'batch_size': 10,
    'gradient_accumulation_steps': 1,
    'device': 'cpu',#'cuda:0',
    'output_dir': './model_path/proact_models', # output folder
    'best_name': 'best_als_predictor',  # output model name
    'max_len_seq': 40,
    'max_age': 84,
    'age_year': True,
    'age_symbol': None,
    'min_visit': 3
}

pretrain_model_path = 'model_path/proact_model'  # pretrained MLM path

In [14]:
BertVocab = load_obj(file_config['vocab'])
ageVocab, _ = age_vocab(max_age=global_params['max_age'], symbol=global_params['age_symbol'])

In [15]:
ageVocab = {k:v for k,v in ageVocab.items() if v <=85 }


In [16]:
# re-format label token
def format_label_vocab(token2idx):
    token2idx = token2idx.copy()
    del token2idx['PAD']
    del token2idx['SEP']
    del token2idx['CLS']
    del token2idx['MASK']
    token = list(token2idx.keys())
    labelVocab = {}
    for i,x in enumerate(token):
        labelVocab[x] = i
    return labelVocab

labelVocab = format_label_vocab(BertVocab['token2idx'])

In [17]:
model_config = {
    'vocab_size': len(BertVocab['token2idx'].keys()), # number of disease + symbols for word embedding
    'hidden_size': 288, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(ageVocab.keys()), # number of vocab for age embedding
    'max_position_embedding': global_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 6, # number of multi-head attention layers required
    'num_attention_heads': 12, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 512, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
}

feature_dict = {
    'word':True,
    'seg':True,
    'age':True,
    'position': True,
    'creatinine':True,
}

In [18]:
class BertConfig(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings = config.get('max_position_embedding'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')

In [19]:
train = pd.read_parquet(file_config['train'])
train = train.rename(columns={'FRS_diag':'code', 'target':'label', 'PID':'patid'})
Dset = NextVisit(token2idx=BertVocab['token2idx'], label2idx=labelVocab, age2idx=ageVocab, dataframe=train, max_len=global_params['max_len_seq'])
trainload = DataLoader(dataset=Dset, batch_size=global_params['batch_size'], shuffle=True, num_workers=3)

In [20]:
test = pd.read_parquet(file_config['test'])
test = test.rename(columns={'FRS_diag':'code', 'target':'label', 'PID':'patid'})

Dset = NextVisit(token2idx=BertVocab['token2idx'], label2idx=labelVocab, age2idx=ageVocab, dataframe=test, max_len=global_params['max_len_seq'])
testload = DataLoader(dataset=Dset, batch_size=global_params['batch_size'], shuffle=False, num_workers=3)

In [21]:
# del model
conf = BertConfig(model_config)
model = BertForMultiLabelPrediction(conf, num_labels=len(labelVocab.keys()), feature_dict=feature_dict)

In [22]:
def load_model(path, model):
    # load pretrained model and update weights
    pretrained_dict = torch.load(path)
    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    return model

mode = load_model(pretrain_model_path, model)

In [23]:
model = model.to(global_params['device'])
optim = optimiser.adam(params=list(model.named_parameters()), config=optim_config)

t_total value of -1 results in schedule not being applied


In [24]:
import sklearn
def precision(logits, label):
    sig = nn.Sigmoid()
    output=sig(logits)
    label, output=label.cpu(), output.detach().cpu()
    tempprc= sklearn.metrics.average_precision_score(label.numpy(),output.numpy(), average='samples')
    return tempprc, output, label

def precision_test(logits, label):
    sig = nn.Sigmoid()
    output=sig(logits)
    tempprc= sklearn.metrics.average_precision_score(label.numpy(),output.numpy(), average='samples')
    roc = sklearn.metrics.roc_auc_score(label.numpy(),output.numpy(), average='samples')
    return tempprc, roc, output, label,

In [25]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=list(labelVocab.values()))
mlb.fit([[each] for each in list(labelVocab.values())])

MultiLabelBinarizer(classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                             15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                             28, 29, ...])

In [27]:
def train(e):
    model.train()
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt = 0
    for step, batch in enumerate(trainload):
        cnt +=1
        age_ids, input_ids, posi_ids, creatinine,segment_ids, attMask, targets, _ = batch
        # TODO: To predict ALS use case I need to rewrite the target maker
        # it will done by setting a single visit as target -> it should contain one vcalue
        # per function
        targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        creatinine = creatinine.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        targets = targets.to(global_params['device'])
        
        loss, logits = model(input_ids, age_ids, segment_ids, posi_ids,creatinine, attention_mask=attMask, labels=targets)
        
        if global_params['gradient_accumulation_steps'] >1:
            loss = loss/global_params['gradient_accumulation_steps']
        loss.backward()
        
        temp_loss += loss.item()
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        
        if step % 50==0:
            prec, a, b = precision(logits, targets)
            print("epoch: {}\t| Cnt: {}\t| Loss: {}\t| precision: {}".format(e, cnt,temp_loss/50, prec))
            temp_loss = 0
        
        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
            optim.zero_grad()

def evaluation():
    model.eval()
    y = []
    y_label = []
    tr_loss = 0
    for step, batch in enumerate(testload):
        model.eval()
        age_ids, input_ids, posi_ids, creatinine, segment_ids, attMask, targets, _ = batch
        targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)
        
        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        creatinine = creatinine.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        targets = targets.to(global_params['device'])
        
        with torch.no_grad():
            loss, logits = model(input_ids, age_ids, segment_ids, posi_ids,creatinine,attention_mask=attMask, labels=targets)
        logits = logits.cpu()
        targets = targets.cpu()
        
        tr_loss += loss.item()

        y_label.append(targets)
        y.append(logits)

    y_label = torch.cat(y_label, dim=0)
    y = torch.cat(y, dim=0)

    aps, roc, output, label = precision_test(y, y_label)
    return aps, roc, tr_loss

In [28]:
best_pre = 0.0
for e in range(500):
    train(e)
    aps, roc, test_loss = evaluation()
    if aps >best_pre:
        # Save a trained model
        print("** ** * Saving fine - tuned model ** ** * ")
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        output_model_file = os.path.join(global_params['output_dir'],global_params['best_name'])
        create_folder(global_params['output_dir'])

        torch.save(model_to_save.state_dict(), output_model_file)
        best_pre = aps
    print(f'aps : {aps} | test loss : {test_loss} | auc : {roc}')

KeyboardInterrupt: 

In [ ]:
test_r = pd.read_parquet('test.parquet')
test_r = test_r.rename(columns={'FRS_diag':'code', 'target':'label', 'PID':'patid'})

Dset = NextVisit(token2idx=BertVocab['token2idx'], label2idx=labelVocab, age2idx=ageVocab, dataframe=test_r, max_len=global_params['max_len_seq'])
testload = DataLoader(dataset=Dset, batch_size=global_params['batch_size'], shuffle=False, num_workers=3)



In [ ]:
def evaluation():
    y = []
    y_label = []
    for step, batch in enumerate(testload):
        age_ids, input_ids, posi_ids, creatinine, segment_ids, attMask, targets, _ = batch
        targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        creatinine = creatinine.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        targets = targets.to(global_params['device'])

        with torch.no_grad():
            loss, logits = model(input_ids, age_ids, segment_ids, posi_ids,creatinine,attention_mask=attMask, labels=targets)
        logits = logits.cpu()
        targets = targets.cpu()

        y_label.append(targets)
        y.append(logits)

    y_label = torch.cat(y_label, dim=0)
    y = torch.cat(y, dim=0)

    return y, y_label

y, y_label = evaluation()
aps, roc, output, label = precision_test(y, y_label)

In [ ]:
new_y = []
for _y in y:
    _y = _y.numpy()
    pid_i = []
    for idx in range(0, 51, 5):
        q_fun = _y[idx:idx+5]
        max_val = np.max(q_fun)
        inner_res = \
            (q_fun==np.max(q_fun)).astype('int8')
        pid_i.extend(inner_res)
    new_y.append(np.array(pid_i))
new_y = np.array(new_y)




In [ ]:
y = []
for _ in new_y:
    y.append(mlb.inverse_transform(_.reshape(-1, 51))[0])

In [ ]:
pred_tokens = []
for pred in y:
    pred_tokens.append(list(BertVocab['idx2token'][p] for p in pred))
pred_tokens = np.array(pred_tokens)


In [ ]:
test_r['pred'] = [list(i) for i in pred_tokens]

In [ ]:
test_r.head()


In [ ]:
def score_calc(predicted):
    score = 0
    for i in predicted:
        if i=='SEP':
            continue
        func_score = i[i.rfind('_') + 1:]
        func_score = int(func_score)
        score += func_score
    return score

In [ ]:
test_r['pred_score'] = test_r['pred'].apply(score_calc)
test_r['label_score'] = test_r['label'].apply(score_calc)

In [ ]:
test_r.head()


In [ ]:

test_r[['pred_score', 'label_score']]

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(test_r['pred_score'], test_r['label_score'])

In [ ]:
def calc_func_diff(true, pred, q):
    for i in true:
        if i.startswith(f'Q{q}_'):
            true_score = int(i[i.rfind('_') + 1:])
            break
    for i in pred:
        if i.startswith(f'Q{q}_'):
            pred_score = int(i[i.rfind('_') + 1:])
            break
    return true_score-pred_score

for i in range(1,11):
    test_r[f'Q{i}_dff'] = test_r.apply(lambda row: calc_func_diff(true=row['label'], pred=row['pred'], q=i), axis=1)

In [ ]:
test_r.head()

In [ ]:

test_r.to_csv('PROACT_RES_2_APS_63822_ROC836178_.csv')

In [ ]:
aps, roc, output, label

In [ ]:
for col in test_r.columns:
    if '_dff' in col:
        print(col, test_r[col].abs().mean())
print('MAE', mean_absolute_error(test_r['pred_score'], test_r['label_score']))


